# Iphone 12

In [85]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

df = pd.read_csv('/Users/xueyuanli/Desktop/DS-team/Asurion_data_iPhone_v3.csv', parse_dates=['weeks_monday','month_year'])
df_12=df[(df['phone_generation'] == 'iphone 12')]

In [86]:
df_12=df_12[['phone_size', 'phone_color',  'claim' ]]
df_12['phone_size'] = df_12['phone_size'].str.replace('gb', '')

print(df_12)

     phone_size phone_color  claim
3027        128       black     14
3028        128       black     43
3029        128       black     57
3030        128       black     65
3031        128       black     59
...         ...         ...    ...
6241        512      silver      7
6242        512      silver      9
6243        512      silver      5
6244        512      silver      8
6245        512      silver     12

[3219 rows x 3 columns]


In [87]:
from sklearn.model_selection import train_test_split

X = df_12[['phone_size','phone_color']]
y = df_12['claim']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [88]:
print(X_train)

     phone_size phone_color
5457        128        gold
3393        128         red
4987        128      silver
3056        128       black
5713        256        blue
...         ...         ...
4122         64       white
4157        128       black
4321        128      purple
3887         64       green
6201        512      silver

[2575 rows x 2 columns]


In [89]:
X_train = pd.get_dummies(X_train, columns=['phone_size','phone_color'])
X_test = pd.get_dummies(X_test, columns=['phone_size','phone_color'])

In [90]:
print(X_train.shape)

(2575, 13)


In [91]:
print(X_test.shape)
print(X_test)

(644, 13)
      phone_size_128  phone_size_256  phone_size_512  phone_size_64  \
3325               1               0               0              0   
3129               1               0               0              0   
4859               1               0               0              0   
4205               1               0               0              0   
5996               0               1               0              0   
...              ...             ...             ...            ...   
3388               1               0               0              0   
3162               1               0               0              0   
4469               0               0               0              1   
5360               1               0               0              0   
3959               0               0               0              1   

      phone_color_black  phone_color_blue  phone_color_gold  phone_color_gray  \
3325                  0                 0               

In [92]:
import pandas as pd
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyRegressor
from sklearn.utils import shuffle
import numpy as np


reg = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric = None)
# Train and test the models
models,predictions = reg.fit(X_train, X_test, y_train, y_test)

print(models)

100%|███████████████████████████████████████████| 42/42 [01:54<00:00,  2.72s/it]

                               Adjusted R-Squared  R-Squared   RMSE  \
Model                                                                 
RandomForestRegressor                        0.40       0.41  69.05   
BaggingRegressor                             0.40       0.41  69.07   
MLPRegressor                                 0.40       0.41  69.10   
DecisionTreeRegressor                        0.40       0.41  69.13   
ExtraTreeRegressor                           0.40       0.41  69.13   
ExtraTreesRegressor                          0.40       0.41  69.13   
XGBRegressor                                 0.40       0.41  69.13   
LGBMRegressor                                0.40       0.41  69.13   
HistGradientBoostingRegressor                0.40       0.41  69.13   
GaussianProcessRegressor                     0.40       0.41  69.13   
GradientBoostingRegressor                    0.40       0.41  69.18   
PoissonRegressor                             0.37       0.38  70.82   
LassoL

# Predict the claim by phone color and phone size

# Random Forest

In [97]:
from sklearn.ensemble import RandomForestRegressor

RFR_model=RandomForestRegressor()
RFR_model.fit(X_train, y_train)

pred = RFR_model.predict(X_test)

In [98]:
def wmape(actual, forecast):
    # we take two series and calculate an output a wmape from it, not to be used in a grouping function

    # make a series called mape
    se_mape = abs(actual-forecast)/actual

    # get a float of the sum of the actual
    ft_actual_sum = actual.sum()

    # get a series of the multiple of the actual & the mape
    se_actual_prod_mape = actual * se_mape

    # summate the prod of the actual and the mape
    ft_actual_prod_mape_sum = se_actual_prod_mape.sum()

    # float: wmape of forecast
    ft_wmape_forecast = ft_actual_prod_mape_sum / ft_actual_sum

    # return a float
    return ft_wmape_forecast

In [99]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, pred)
print('MSE:', mse)

wmape=wmape(y_test, pred)
print("WMAPE: {:.2%}".format(wmape))

MSE: 4776.2408446693225
WMAPE: 59.51%


# xgboost

In [100]:
from sklearn.linear_model import LinearRegression
import xgboost as xgb
#model = LinearRegression()
model=xgb.XGBRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [101]:
from sklearn.metrics import mean_squared_error, r2_score

print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
print('R2 score: %.2f' % r2_score(y_test, y_pred))

Mean squared error: 4779.20
R2 score: 0.41


In [102]:
def wmape(actual, forecast):
    # we take two series and calculate an output a wmape from it, not to be used in a grouping function

    # make a series called mape
    se_mape = abs(actual-forecast)/actual

    # get a float of the sum of the actual
    ft_actual_sum = actual.sum()

    # get a series of the multiple of the actual & the mape
    se_actual_prod_mape = actual * se_mape

    # summate the prod of the actual and the mape
    ft_actual_prod_mape_sum = se_actual_prod_mape.sum()

    # float: wmape of forecast
    ft_wmape_forecast = ft_actual_prod_mape_sum / ft_actual_sum

    # return a float
    return ft_wmape_forecast


In [103]:

wmape=wmape(y_test, y_pred)

print("WMAPE: {:.2%}".format(wmape))

WMAPE: 59.47%


# DecisionTreeRegressor

In [104]:
from sklearn.tree import DecisionTreeRegressor

DT_model=DecisionTreeRegressor()
DT_model.fit(X_train, y_train)

pred = DT_model.predict(X_test)

In [105]:
def wmape(actual, forecast):
    # we take two series and calculate an output a wmape from it, not to be used in a grouping function

    # make a series called mape
    se_mape = abs(actual-forecast)/actual

    # get a float of the sum of the actual
    ft_actual_sum = actual.sum()

    # get a series of the multiple of the actual & the mape
    se_actual_prod_mape = actual * se_mape

    # summate the prod of the actual and the mape
    ft_actual_prod_mape_sum = se_actual_prod_mape.sum()

    # float: wmape of forecast
    ft_wmape_forecast = ft_actual_prod_mape_sum / ft_actual_sum

    # return a float
    return ft_wmape_forecast



In [106]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, pred)
print('MSE:', mse)

wmape=wmape(y_test, pred)
print("WMAPE: {:.2%}".format(wmape))

MSE: 4779.199351494688
WMAPE: 59.47%


# Predict the claim by phone color

> xgboost model

In [107]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# load the dataset
df = pd.read_csv('/Users/xueyuanli/Desktop/DS-team/Asurion_data_iPhone_v3.csv')
df=df[(df['phone_generation'] == 'iphone 12')]
df=df[[ 'phone_color',  'claim' ]]
#df['phone_size'] = df['phone_size'].str.replace('gb', '')


# preprocess the data
one_hot = pd.get_dummies(df['phone_color'])
df = pd.concat([df, one_hot], axis=1)
#df['phone_size'] = df['phone_size'].str.replace('gb', '').astype(float)

df=df.drop(['phone_color'], axis=1)
print(df)

      claim  black  blue  gold  gray  green  purple  red  silver  white
3027     14      1     0     0     0      0       0    0       0      0
3028     43      1     0     0     0      0       0    0       0      0
3029     57      1     0     0     0      0       0    0       0      0
3030     65      1     0     0     0      0       0    0       0      0
3031     59      1     0     0     0      0       0    0       0      0
...     ...    ...   ...   ...   ...    ...     ...  ...     ...    ...
6241      7      0     0     0     0      0       0    0       1      0
6242      9      0     0     0     0      0       0    0       1      0
6243      5      0     0     0     0      0       0    0       1      0
6244      8      0     0     0     0      0       0    0       1      0
6245     12      0     0     0     0      0       0    0       1      0

[3219 rows x 10 columns]


In [108]:
# split the data into features and target
X = df.drop('claim', axis=1)
y = df['claim']

# train a linear regression model
#model = LinearRegression()

import xgboost as xgb

model=xgb.XGBRegressor()
model.fit(X, y)

# make a prediction for a new phone color (e.g. 'pink')
new_color = pd.DataFrame({'black': 1, 'blue': 0, 'gold': 0, 'gray': 0,
                           'green': 0, 'purple': 0, 'red': 0, 'silver': 0,
                           'white': 0}, index=[0])
prediction = model.predict(new_color)

print(f"The predicted claim volume for the new phone color is {prediction[0]}")

The predicted claim volume for the new phone color is 86.86316680908203


In [109]:
# make predictions for the training data
y_pred = model.predict(X)

# compute the evaluation metrics
mse = mean_squared_error(y, y_pred)
rmse = mean_squared_error(y, y_pred, squared=False)
r2 = r2_score(y, y_pred)

print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R-squared score: {r2:.2f}")

MSE: 5659.67
RMSE: 75.23
R-squared score: 0.28


In [110]:
def wmape(actual, forecast):
    # we take two series and calculate an output a wmape from it, not to be used in a grouping function

    # make a series called mape
    se_mape = abs(actual-forecast)/actual

    # get a float of the sum of the actual
    ft_actual_sum = actual.sum()

    # get a series of the multiple of the actual & the mape
    se_actual_prod_mape = actual * se_mape

    # summate the prod of the actual and the mape
    ft_actual_prod_mape_sum = se_actual_prod_mape.sum()

    # float: wmape of forecast
    ft_wmape_forecast = ft_actual_prod_mape_sum / ft_actual_sum

    # return a float
    return ft_wmape_forecast





In [111]:
wmape=wmape(y, y_pred)

print("WMAPE: {:.2%}".format(wmape))

WMAPE: 68.35%


# Predict the claim by phone size

> xgboost model

In [112]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# load the dataset
df = pd.read_csv('/Users/xueyuanli/Desktop/DS-team/Asurion_data_iPhone_v3.csv')
df=df[(df['phone_generation'] == 'iphone 12')]
df=df[[ 'phone_size',  'claim' ]]
df['phone_size'] = df['phone_size'].str.replace('gb', '')


# preprocess the data
one_hot = pd.get_dummies(df['phone_size'])
df = pd.concat([df, one_hot], axis=1)
df['phone_size'] = df['phone_size'].str.replace('gb', '').astype(float)

df=df.drop(['phone_size'], axis=1)
print(df)

      claim  128  256  512  64
3027     14    1    0    0   0
3028     43    1    0    0   0
3029     57    1    0    0   0
3030     65    1    0    0   0
3031     59    1    0    0   0
...     ...  ...  ...  ...  ..
6241      7    0    0    1   0
6242      9    0    0    1   0
6243      5    0    0    1   0
6244      8    0    0    1   0
6245     12    0    0    1   0

[3219 rows x 5 columns]


In [113]:
# split the data into features and target
X = df.drop('claim', axis=1)
y = df['claim']

# train a linear regression model
#model = LinearRegression()
model=xgb.XGBRegressor()
model.fit(X, y)

# make a prediction for a new phone color (e.g. 'pink')
new_color = pd.DataFrame({'128': 1, '256': 0, '512': 0, '64': 0}, index=[0])
prediction = model.predict(new_color)

print(f"The predicted claim volume for the new phone size is {prediction[0]}")

The predicted claim volume for the new phone size is 90.40621185302734


In [114]:
# make predictions for the training data
y_pred = model.predict(X)

# compute the evaluation metrics
mse = mean_squared_error(y, y_pred)
rmse = mean_squared_error(y, y_pred, squared=False)
r2 = r2_score(y, y_pred)

print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R-squared score: {r2:.2f}")

MSE: 7519.19
RMSE: 86.71
R-squared score: 0.04


In [115]:
def wmape(actual, forecast):
    # we take two series and calculate an output a wmape from it, not to be used in a grouping function

    # make a series called mape
    se_mape = abs(actual-forecast)/actual

    # get a float of the sum of the actual
    ft_actual_sum = actual.sum()

    # get a series of the multiple of the actual & the mape
    se_actual_prod_mape = actual * se_mape

    # summate the prod of the actual and the mape
    ft_actual_prod_mape_sum = se_actual_prod_mape.sum()

    # float: wmape of forecast
    ft_wmape_forecast = ft_actual_prod_mape_sum / ft_actual_sum

    # return a float
    return ft_wmape_forecast




In [116]:
wmape=wmape(y, y_pred)

print("WMAPE: {:.2%}".format(wmape))

WMAPE: 81.28%


# iphone 14
> xgboost

In [144]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# load the dataset
df = pd.read_csv('/Users/xueyuanli/Desktop/DS-team/Asurion_data_iPhone_v3.csv')
df=df[(df['phone_generation'] == 'iphone 14')]
df=df[[ 'phone_color',  'claim' ]]

# preprocess the data
one_hot = pd.get_dummies(df['phone_color'])
df = pd.concat([df, one_hot], axis=1)
#df['phone_size'] = df['phone_size'].str.replace('gb', '').astype(float)

df=df.drop(['phone_color'], axis=1)
print(df)

      claim  black  blue  gold  purple  red  silver  white
7907      7      1     0     0       0    0       0      0
7908     11      1     0     0       0    0       0      0
7909     15      1     0     0       0    0       0      0
7910     25      1     0     0       0    0       0      0
7911     20      1     0     0       0    0       0      0
...     ...    ...   ...   ...     ...  ...     ...    ...
7970     50      0     0     1       0    0       0      0
7971     18      0     0     1       0    0       0      0
7972     68      0     0     0       1    0       0      0
7973     24      0     0     0       1    0       0      0
7974      1      0     0     0       0    0       1      0

[68 rows x 8 columns]


In [145]:
# split the data into features and target
X = df.drop('claim', axis=1)
y = df['claim']

# train a linear regression model
model=xgb.XGBRegressor()
#model=RandomForestRegressor()
#model = LinearRegression()
model.fit(X, y)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [146]:
# make predictions for the training data
y_pred = model.predict(X)

# compute the evaluation metrics
mse = mean_squared_error(y, y_pred)
rmse = mean_squared_error(y, y_pred, squared=False)
r2 = r2_score(y, y_pred)

print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R-squared score: {r2:.2f}")

MSE: 145.09
RMSE: 12.05
R-squared score: 0.33


In [147]:
def wmape(actual, forecast):
    # we take two series and calculate an output a wmape from it, not to be used in a grouping function

    # make a series called mape
    se_mape = abs(actual-forecast)/actual

    # get a float of the sum of the actual
    ft_actual_sum = actual.sum()

    # get a series of the multiple of the actual & the mape
    se_actual_prod_mape = actual * se_mape

    # summate the prod of the actual and the mape
    ft_actual_prod_mape_sum = se_actual_prod_mape.sum()

    # float: wmape of forecast
    ft_wmape_forecast = ft_actual_prod_mape_sum / ft_actual_sum

    # return a float
    return ft_wmape_forecast



In [148]:
wmape=wmape(y, y_pred)

print("WMAPE: {:.2%}".format(wmape))

WMAPE: 65.47%
